In [4]:
!git clone https://github.com/thangnch/MiAI_StableDiffusion_Flask_Text2Image
%cd MiAI_StableDiffusion_Flask_Text2Image/
!pip -q install -r setup.txt

Cloning into 'MiAI_StableDiffusion_Flask_Text2Image'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 3), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 80.09 KiB | 2.86 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/MiAI_StableDiffusion_Flask_Text2Image
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.5 MB/s eta 0:00:00


In [7]:
!pip -q install pyngrok

In [5]:
import torch
from diffusers import StableDiffusionPipeline

# Định nghĩa tham số
rand_seed = torch.manual_seed(42)
NUM_INFERENCE_STEPS = 50
GUIDANCE_SCALE = 0.75
HEIGHT = 512
WIDTH = 512

# Danh sách model
model_list = ["nota-ai/bk-sdm-small",
              "CompVis/stable-diffusion-v1-4",
              "runwayml/stable-diffusion-v1-5",
              "prompthero/openjourney",
              "hakurei/waifu-diffusion",
              "stabilityai/stable-diffusion-2-1",
              "dreamlike-art/dreamlike-photoreal-2.0"
              ]


def create_pipeline(model_name = model_list[1]):
    # Nếu máy có GPU CUDA
    if torch.cuda.is_available():
        print("Using GPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype = torch.float16,
            use_safetensors = True
        ).to("cuda")
    elif torch.backends.mps.is_available():
        print("Using MPS")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            use_safetensors=True
        ).to("mps")
    else:
        print("Using CPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float32,
            use_safetensors=True
        )
    return pipeline

def text2img(prompt, pipeline):
    images = pipeline(
        prompt,
        guidance_scale = GUIDANCE_SCALE,
        num_inference_steps = NUM_INFERENCE_STEPS,
        generator = rand_seed,
        num_images_per_request = 1,
        height = HEIGHT,
        width = WIDTH
    ).images

    return images[0]



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
from flask import Flask, request, render_template
from text2img_model import create_pipeline, text2img
from pyngrok import ngrok, conf

from google.colab import userdata


# Khởi tạo Flask app
app =  Flask(__name__)

# Định nghĩa tham số
IMAGE_PATH  = "static/output.jpg"
conf.get_default().auth_token = userdata.get('ngrok_token')

public_url = ngrok.connect(8888).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


# Khởi tạo pipeline
pipeline = create_pipeline()

@app.route("/", methods = ['GET', 'POST'])
def index():
    # Kiểm tra xem có phải người dùng view trang web không
    if request.method == "GET":
        # Trả về giao diện trang web
        return render_template("index.html")
    else:
        # Xử lý việc người dùng submit prompt-> sinh ảnh -> trả về
        user_input = request.form["prompt"]

        print("Start gen....")
        im = text2img(user_input, pipeline)
        print("Finish gen....")
        im.save(IMAGE_PATH)

        return render_template("index.html", image_url = IMAGE_PATH)

if __name__ =='__main__':
    app.run(debug=False, host='0.0.0.0', port=8888, use_reloader=False)




 * ngrok tunnel "https://be11-34-73-97-133.ngrok-free.app" -> "http://127.0.0.1:5000/"
Using GPU


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8888
 * Running on http://172.28.0.12:8888
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Aug/2024 03:12:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2024 03:12:15] "GET /favicon.ico HTTP/1.1" 404 -


Start gen....


...

Start gen....


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [24/Aug/2024 03:12:57] "POST / HTTP/1.1" 200 -


Finish gen....


INFO:werkzeug:127.0.0.1 - - [24/Aug/2024 03:12:58] "GET /static/output.jpg HTTP/1.1" 200 -


In [3]:
import getpass
import os
import threading

from flask import Flask


print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

# # Start the Flask server in a new thread
# threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
app.run()


Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········
 * ngrok tunnel "https://7165-34-125-100-219.ngrok-free.app" -> "http://127.0.0.1:5000/"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Aug/2024 02:53:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2024 02:53:44] "GET /favicon.ico HTTP/1.1" 404 -


In [2]:
!pip install pyngrok
